Açıklama: Eğitimi colab üzerinden yaptım . Drivedan veriyi çekip colab üzerinden unzip felan yapıyordum ve 5 farklı hesap kullanıyordum o yüzden kod ve adresler çok karışıktı.Kodu düzenleyip kaggle'a aktardım.

En iyi modelin eğitiminde 

epoch5 : f1 0.925  

epoch10 : f1 0.9393

epoch14 : f1 0.9422 ---> bunu yolladım

Colab kodunuda paylaştım oradan çıktılara bakabilirsiniz. Colab kodunda eğitim 2. epoch'tan başladı çünkü  eğitirken 2. epochta yanlışlıkla eğitimi durdurdum. 1 epoch'taki kayıttan devam ettirdim .

In [ ]:

import pandas as pd
import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torch import nn, optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, f1_score
from PIL import Image
import os
from sklearn.preprocessing import LabelEncoder
from torch.optim.lr_scheduler import ReduceLROnPlateau

mapping

In [ ]:
csv_path = "/kaggle/input/datathon-ai-qualification-round/train_data.csv"
data = pd.read_csv(csv_path)
label_mapping = {'Ankara': 0, 'Istanbul': 1, 'Izmir': 2}
data['city'] = data['city'].map(label_mapping)

In [ ]:
train_df, test_df = train_test_split(data, test_size=0.2, stratify=data['city'], random_state=42)

In [ ]:
train_csv_path = "/kaggle/working/train.csv"
test_csv_path = "/kaggle/working/test.csv"

train_df.to_csv(train_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

Augmentation kodu: horizanlar flip , color jitter ve random crop içerir


In [ ]:
import os
import pandas as pd
from PIL import Image


images_dir = "/kaggle/input/datathon-ai-qualification-round/train/train"
train_csv = "/kaggle/working/train.csv"
test_csv = "/kaggle/working/test.csv"
augmented_dir = "/kaggle/working/augmented_train"
augmented_csv = "/kaggle/working/augmented_train.csv"

os.makedirs(augmented_dir, exist_ok=True)
train_df = pd.read_csv(train_csv)


augmented_data = []

for index, row in train_df.iterrows():
    filename = row['filename']
    city = row['city']
    
    image_path = os.path.join(images_dir, filename)
    
    if os.path.exists(image_path):
        img = Image.open(image_path)
        
        original_save_path = os.path.join(augmented_dir, filename)
        img.save(original_save_path)
        augmented_data.append({"filename": filename, "city": city})
        
        flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
        flipped_filename = f"flipped_{filename}"
        flipped_save_path = os.path.join(augmented_dir, flipped_filename)
        flipped_img.save(flipped_save_path)
        augmented_data.append({"filename": flipped_filename, "city": city})
    else:
        print(f"Warning: {filename} not found in {images_dir}")


augmented_df = pd.DataFrame(augmented_data)
augmented_df.to_csv(augmented_csv, index=False)

print(f"Augmentation işlemi tamamlandı. Veriler {augmented_dir} klasörüne ve {augmented_csv} dosyasına kaydedildi.")


In [ ]:
##test kumesinide aynı klasore tasidim
import os
import pandas as pd
from PIL import Image
test_csv = "/kaggle/working/test.csv"
test_df = pd.read_csv(test_csv)

def copy_test_images(df, images_dir, augmented_dir):
    for index, row in df.iterrows():
        filename = row['filename']
        city = row['city']

        image_path = os.path.join(images_dir, filename)

        if os.path.exists(image_path):
            img = Image.open(image_path)
            save_path = os.path.join(augmented_dir, filename)
            img.save(save_path)
        else:
            print(f"Warning: {filename} not found in {images_dir}")

copy_test_images(test_df, images_dir, augmented_dir)

print(f"Test verilerindeki görüntüler {augmented_dir} klasörüne kopyalandı.")


In [ ]:
import os
import pandas as pd
from PIL import Image, ImageEnhance
import random

images_dir = "/content/augmented_train"
train_csv = "/content/drive/MyDrive/Datathonaİ/datasets/dataset/augmented_train.csv"
augmented_dir = "/content/augmented_train"
augmented_csv = "/content/drive/MyDrive/Datathonaİ/datasets/dataset/augmented_trainbg.csv"


os.makedirs(augmented_dir, exist_ok=True)


train_df = pd.read_csv(train_csv)

augmented_data = []

for index, row in train_df.iterrows():
    filename = row['filename']
    city = row['city']

    image_path = os.path.join(images_dir, filename)

    if os.path.exists(image_path):
        img = Image.open(image_path).convert("RGB")  

        """
        original_save_path = os.path.join(augmented_dir, filename)
        img.save(original_save_path)
        """
        augmented_data.append({"filename": filename, "city": city})


        brightness = ImageEnhance.Brightness(img).enhance(random.uniform(0.8, 1.2))
        contrast = ImageEnhance.Contrast(brightness).enhance(random.uniform(0.8, 1.2))
        saturation = ImageEnhance.Color(contrast).enhance(random.uniform(0.8, 1.2))
        jittered_img = ImageEnhance.Brightness(saturation).enhance(random.uniform(0.8, 1.2))

        jittered_filename = f"jittered_{filename}"
        jittered_save_path = os.path.join(augmented_dir, jittered_filename)
        jittered_img.save(jittered_save_path)
        augmented_data.append({"filename": jittered_filename, "city": city})


        width, height = img.size
        crop_size = 540
        if width >= crop_size and height >= crop_size:
            left = random.randint(0, width - crop_size)
            top = random.randint(0, height - crop_size)
            cropped_img = img.crop((left, top, left + crop_size, top + crop_size))
            resized_img = cropped_img.resize((640, 640))

            cropped_filename = f"cropped_{filename}"
            cropped_save_path = os.path.join(augmented_dir, cropped_filename)
            resized_img.save(cropped_save_path)
            augmented_data.append({"filename": cropped_filename, "city": city})
        else:
            print(f"Warning: {filename} boyutları kırpma için uygun değil.")
    else:
        print(f"Warning: {filename} not found in {images_dir}")


augmented_df = pd.DataFrame(augmented_data)
augmented_df.to_csv(augmented_csv, index=False)

print(f"Augmentation işlemi tamamlandı. Veriler {augmented_dir} klasörüne ve {augmented_csv} dosyasına kaydedildi.")


**Preprocessing**

In [ ]:

class CityDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['filename']
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.dataframe.iloc[idx]['city']

        if self.transform:
            image = self.transform(image)

        return image, label


img_dir = "/kaggle/working/augmented_train"


In [ ]:

label_mapping = {'Ankara': 0, 'Istanbul': 1, 'Izmir': 2}
##train test spliti her seferinde çalıştırmadım hesap değiştirdiğim zamanlar çalıştırdım
#train_df, test_df = train_test_split(data, test_size=0.2, stratify=data['city'], random_state=42)

train_csv_path  = "/content/drive/MyDrive/Datathonaİ/datasets/dataset/augmented_trainbg.csv"
test_csv_path  = "/kaggle/working/test.csv"

train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)
# yorum satırına alınan yerleri hesap değiştireceğim zamanlar yani yeni runtime açtığımada çalıştırıyordum
#train_df['city'] = train_df['city'].map(label_mapping)

##train_df.to_csv(train_csv_path, index=False)
#test_df.to_csv(test_csv_path, index=False)

transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])




**Aşağıdaki kod en iyi sonuç aldığım model'in eğitim kodudur**

In [ ]:
train_dataset = CityDataset(train_df, img_dir, transform=transform)
test_dataset = CityDataset(test_df, img_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


model = models.efficientnet_b3(pretrained=True)  
num_classes = len(label_mapping)

# Classifier'ı değiştirme
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3),  # Overfitting'i önlemek için Dropout
    nn.Linear(model.classifier[1].in_features, num_classes)
)
  # Son lineer katmanı değiştiriyoruz

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#GPU memoryinin yetmesi için bu 4 katmanı dondurdum colab t4 gpu tam yetti
for name, param in model.named_parameters():
    if "features.0" in name or "features.1" in name or "features.2" in name or "features.3" in name :
        param.requires_grad = False


for name, param in model.named_parameters():
    print(f"{name}: {'Trainable' if param.requires_grad else 'Frozen'}")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=2, verbose=True)

In [ ]:
def save_checkpoint(epoch, model, optimizer, scheduler, path="/kaggle/working/efficentnetb3.pth"):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() 
    }, path)


def load_checkpoint(model, optimizer, scheduler, path="/content/efficentnet2.pth"):
    if os.path.exists(path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint['model_state_dict']) #cpu da  calisacagi zaman map locarion = cpu eklemek gerekiyor
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict']) 
        start_epoch = checkpoint['epoch']
        print(f"Checkpoint yükleniyor: {path}, Epoch: {start_epoch}")
        return start_epoch
    else:
        print("Checkpoint bulunamadı. Eğitim sıfırdan başlıyor.")
        return 0


**Kodda f1 scoru en yüksek olan modeli kaydediyor ve lr scheduler f1 scoruna göre lr yi ayarlıyor.**

In [ ]:
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs=50, checkpoint_interval=3, start_epoch=0):
    best_macro_f1 = 0.0
    for epoch in range(start_epoch, epochs):

        model.train()
        print("--------------------------------------")
        running_loss = 0.0
        i = 0
        for images, labels in train_loader:

            if (i+1) % 100 == 0:
                print(i)
            i += 1

            images, labels = images.to(device), labels.to(device).long()

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

        # test kümesinde dogrulama
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                y_true.extend(labels.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())

        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
        for city, idx in label_mapping.items():
            print(f"Class: {city}, Precision: {precision[idx]:.4f}, Recall: {recall[idx]:.4f}, F1: {f1[idx]:.4f}")

        f1_macro = f1_score(y_true, y_pred, average='macro')
        print(f"Macro F1 Score: {f1_macro:.4f}")

        
        scheduler.step(f1_macro)

        if f1_macro > best_macro_f1:
            best_macro_f1 = f1_macro
            print(f"Yeni en iyi Macro F1 bulundu: {best_macro_f1:.4f}, modeli kaydediyorum...")
            save_checkpoint(epoch + 1, model, optimizer,scheduler)
        print("--------------------------------------")


start_epoch = load_checkpoint(model, optimizer, scheduler,  path="/content/drive/MyDrive/Datathonaİ/datasets/dataset/efficentnet5.pth")
model.to(device)

**En iyi modelin en iyi halini alması 14 epoch sürdü**

In [ ]:
train_model(model, train_loader, criterion, optimizer, scheduler, start_epoch=start_epoch, epochs=25)

Modeli tekrar değerlendiriyoruz kontrol amaçlı

In [ ]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())


precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
for city, idx in label_mapping.items():
    print(f"Class: {city}, Precision: {precision[idx]:.4f}, Recall: {recall[idx]:.4f}, F1: {f1[idx]:.4f}")

f1_macro = f1_score(y_true, y_pred, average='macro')
print(f"Macro F1 Score: {f1_macro:.4f}")

**CSV dosyasına model tahmini yazdırma kodu**

**Bu kodu modeli bilgisayara indirip cpu üzerinden çalıştırdım .**

In [ ]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch


class CityDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['filename']
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, img_name  .

test_csv_path = "test.csv"
img_dir = "C:/Users/Berke/Desktop/dataset/test/test/"

test_df = pd.read_csv(test_csv_path)

transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = CityDataset(test_df, img_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

model.eval()
test2_preds = []

with torch.no_grad():
    for images, filenames in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

      
        predicted_cities = [city for idx in preds.cpu().numpy() for city, label_idx in label_mapping.items() if label_idx == idx]
        test2_preds.extend(zip(filenames, predicted_cities))

predictions_df = pd.DataFrame(test2_preds, columns=['filename', 'city'])
predictions_df.to_csv("C:/Users/Berke/Desktop/dataset/efficentnet3.csv", index=False)

print("Tahminler başarıyla CSV dosyasına kaydedildi!")

